<a href="https://colab.research.google.com/github/Rachna0604/Devrev/blob/main/FinalDevrev.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk==3.4.5
!pip install torch
!pip install transformers

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')
import pandas as pd
#for pre processing
import csv,numpy 
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

In [ ]:
# accessing g drive
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


#1

Predicting paragraphs from question and theme

In [ ]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import pandas as pd

def predict_answer(question, paragraphs, theme):
    # Filter the paragraphs to only include those with the correct theme
    theme_paragraphs = [p['text'] for p in paragraphs if p['theme'] == theme]
    
    # Create a TfidfVectorizer to convert the paragraphs into numerical vectors
    vectorizer = TfidfVectorizer()
    X = vectorizer.fit_transform(theme_paragraphs)
    
    # Convert the question into a numerical vector
    question_vector = vectorizer.transform([question])
    
    # Calculate the cosine similarity between the question and each paragraph
    similarity = cosine_similarity(question_vector, X)
    
    # Find the paragraph with the highest similarity score
    max_similarity_index = similarity.argmax()
    max_similarity_score = similarity[0, max_similarity_index]
    
    # If the maximum similarity score is above a certain threshold, return the paragraph
    if max_similarity_score > 0.05:
        return theme_paragraphs[max_similarity_index]
    else:
        return None


In [ ]:
question = "What is the capital of France?"
paragraphs = [
    {'text': 'Paris is the capital of France', 'theme': 'Geography'},
    {'text': 'Lyon is a city in France', 'theme': 'Geography'},
    {'text': 'The Pythagorean theorem states that a^2 + b^2 = c^2', 'theme': 'Mathematics'},
]

# Predict the answer to the question
answer = predict_answer(question, paragraphs, 'Geography')

# Print the answer
if answer:
    print(answer)
else:
    print("No suitable paragraph found")

Saving the paragraphs,questions and themes in the way it can be passed through the function predict_answer

In [ ]:
path = '/content/drive/MyDrive/DevRev/train_data.csv'
data = pd.read_csv(path)
Paragraphs = []
for para in data['Paragraph']:
    if para not in Paragraphs:
        Paragraphs.append(para)
questions = []
for q in data['Question']:
    if q not in questions:
          questions.append(q)
themes = []
for t in data['Theme']:
    if t not in themes:
        themes.append(t)
PT = []
Themes = []
for element in Paragraphs:
    Themes.append(data['Theme'][data.loc[data['Paragraph'] == element]['Theme'].index[0]])
for i in range(len(Themes)):
    PT.append({'text': Paragraphs[i], 'theme': Themes[i]})
question = questions[0]
answer = predict_answer(question, PT, 'Beyoncé')
if answer:
    print(answer)
else:
    print("No suitable paragraph found")

#2

Traning a model    
**this has no continuation with the above code , it's a seperate method tried 

In [ ]:
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Load the data from the CSV file
df = pd.read_csv("/content/drive/MyDrive/DevRev/train_data.csv")

# Preprocess the data
df["question"] = df["Question"].str.lower()
df["paragraph"] = df["Paragraph"].str.lower()

# Split the data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(df[["question", "paragraph"]], df["Theme"], test_size=0.2)

# Vectorize the data using a TF-IDF vectorizer
vectorizer = TfidfVectorizer()
X_train_vectors = vectorizer.fit_transform(X_train["question"])
X_test_vectors = vectorizer.transform(X_test["question"])

# Train a support vector machine (SVM) classifier
clf = SVC()
clf.fit(X_train_vectors, y_train)

# Predict the theme labels for the test data
y_pred = clf.predict(X_test_vectors)


# Calculate the accuracy of the model
accuracy = accuracy_score(y_test, y_pred)

# Calculate the precision of the model
precision = precision_score(y_test, y_pred, average='macro')

# Calculate the recall of the model
recall = recall_score(y_test, y_pred, average='macro')

# Calculate the F1 score of the model
f1 = f1_score(y_test, y_pred, average='macro')

# Print the results
print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1)


In [ ]:
# Preprocess the new question
new_question = "Who was Hayek's father?"
new_question = new_question.lower()

# Vectorize the new question
new_question_vector = vectorizer.transform([new_question])

# Use the model to make a prediction
prediction = clf.predict(new_question_vector)[0]

# Print the prediction
print("Prediction:", prediction)

#3

Below is another method of approaching the question which I tried earlier, it's not running, but approach can be referred

In [ ]:
def predicting_answer(question, paragraphs):
  #pre process
  question_tokens = preprocess_text(question)
  paragraphs_tokens = [preprocess_text(p) for p in paragraphs]
  #generate embeddings
  question_embeddings = nlp_model(question_tokens)
  paragraphs_embeddings = [nlp_model(p) for p in paragraphs_tokens]
  #compare embeddings i.e. similarity measure
  most_similar_paragraph = find_most_similar(question_embeddings, paragraphs_embeddings)
  #threshold
  if most_similar_paragraph[1] > SIMILARITY_THRESHOLD:
    return paragraphs[most_similar_paragraph[0]]
  else:
    return None

In [ ]:
def preprocess_text(text):
  # Tokenize the text
  tokens = nltk.word_tokenize(text)
  
  # Stem the tokens
  stemmer = PorterStemmer()
  stemmed_tokens = [stemmer.stem(token) for token in tokens]
  
  return stemmed_tokens

# Pre-process the question
question = "What is the capital of France?"
question_tokens = preprocess_text(question)
print(question_tokens)  

# Pre-process a paragraph
paragraph = "Paris is the capital of France. It is located in the northern central part of the country and is home to many famous landmarks, including the Eiffel Tower."
paragraph_tokens = preprocess_text(paragraph)
print(paragraph_tokens)  


In [ ]:
from transformers import BertModel, BertTokenizer
# Initialize the BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
def generate_embeddings(tokens):
  # Encode the text
  input_ids = tokenizer(tokens, return_tensors='pt',padding=True,truncation=True)
  #the output is input tensors in the form of a PyTorch tensor
  print(input_ids)
# Generate embeddings for the question tokens
question_embeddings = generate_embeddings(question_tokens)

# Generate embeddings for the paragraph tokens
paragraph_embeddings = generate_embeddings(paragraph_tokens)


In [ ]:
from scipy import spatial

def cosine_similarity(emb1, emb2):
    similarity = 1 - spatial.distance.cosine(emb1, emb2)
    return similarity

#note
#in this we have to give the axes so for each paragraph in the func defined
#so the word embeddings should be stored in a list, that list hould be passed the function
#like below given hashtags
#paragraph_embeddings = [word_embedding_1, word_embedding_2, word_embedding_3]
#question_embeddings = [word_embedding_4, word_embedding_5, word_embedding_6]

# Calculating the cosine similarity between the two paragraphs
similarity = cosine_similarity(paragraph_embeddings, question_embeddings)

# Determining which paragraph is more similar based on the similarity value
if similarity > 0.8:
    print(paragraph)
else:
    print("Paragraph is not similar")


In [ ]:
#note paragraphs embeddings means there are a lot of paragraphs' embeddings *****
def find_most_similar(question_embeddings, paragraphs_embeddings):
  max_similarity = -1
  most_similar_paragraph = None
  for i, paragraph_embedding in enumerate(paragraphs_embeddings):
    similarity = cosine_similarity(question_embeddings, paragraph_embedding)
    if similarity > max_similarity:
      max_similarity = similarity
      most_similar_paragraph = i
  return most_similar_paragraph, max_similarity